In [50]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
import h5py 
import os
import seaborn as sns
import matplotlib.pyplot as plt
import sys
import glob
from catboost import CatBoostClassifier
from lightgbm.sklearn import LGBMClassifier
from xgboost import XGBClassifier
from sklearn import datasets
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.dummy import DummyClassifier, DummyRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import RFE, RFECV
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression, Ridge, RidgeCV
from sklearn.metrics import (
    make_scorer,
    get_scorer
)
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    ShuffleSplit,
    cross_val_score,
    cross_validate,
    train_test_split,
)
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import (
    OneHotEncoder,
    OrdinalEncoder,
    PolynomialFeatures,
    StandardScaler,
    FunctionTransformer
)
from collections import defaultdict

In [3]:
def read_RICHAI_data(file_paths):
    """Read in all the 2018 RICHAI data
    
    Parameters
    ----------
    file_paths : list of str
        List of file paths of all data sets.
        
    Returns
    -------
    data_files : dict
        Dictionary of HFD data sets     
    """
    data_files = {}
    for file_path in file_paths:
        name = file_path.split("/")[4] + "/" + file_path.split("/")[5].split(".")[0]
        data_files[name] = h5py.File(file_path)
    
    return data_files

In [4]:
def events_to_pandas(dfile):
    """Convert HDF5 events data to Pandas
    
    Parameters
    ----------
    dfile : HDF5 file
        The RICHAI HDF5 file to convert to pandas.
        
    Returns
    -------
    df : pd.DataFrame
        pandas DataFrame with Events data
    """
    
    df = pd.DataFrame()
    
    # event features
    df["run_id"] = dfile["Events"]["run_id"]
    df["burst_id"] = dfile["Events"]["burst_id"]
    df["event_id"] = dfile["Events"]["event_id"]
    df["track_id"] = dfile["Events"]["track_id"]
    df["track_momentum"] = dfile["Events"]["track_momentum"]
    df["chod_time"] = dfile["Events"]["chod_time"]
    df["ring_radius"] = dfile["Events"]["ring_radius"]
    df["ring_centre_pos_x"] = dfile["Events"]["ring_centre_pos"][:, 0]
    df["ring_centre_pos_y"] = dfile["Events"]["ring_centre_pos"][:, 1]
    df["ring_likelihood_pion"] = dfile["Events"]["ring_likelihood"][:, 0]
    df["ring_likelihood_muon"] = dfile["Events"]["ring_likelihood"][:, 1]
    df["ring_likelihood_positron"] = dfile["Events"]["ring_likelihood"][:, 2]
    
    # labels
    mu_off = dfile.attrs["muon_offset"]
    pi_off = dfile.attrs["pion_offset"]
    pos_off = dfile.attrs["positron_offset"]
    entries = dfile.attrs["entries"]
    
    labels = np.zeros(entries, dtype=np.int32)
    labels[mu_off:pi_off] = 0
    labels[pi_off:pos_off] = 1
    labels[pos_off:] = 2
    
    df["label"] = labels
    
    # hit mapping values
    df["first_hit"] = np.array(dfile["HitMapping"])[:-1]  # hit n
    df["last_hit"] = np.array(dfile["HitMapping"])[1:]    # hit n + 1
    df["total_hits"] = df["last_hit"] - df["first_hit"]
    
    return df

In [5]:
def get_string_label(label):
    """Add string label to pandas df (to be used with map)"""
    if label == 0:
        return "muon"
    elif label == 1:
        return "pion"
    elif label == 2:
        return "positron"

In [6]:
def compute_seq_id(hit, or_id=0):
    """Compute the RICH PMT sequence ID"""
    disk_id, pm_id, sc_id, up_dw_id, _ = hit
    
    if or_id < 1:
        seq_id = sc_id * 8 + pm_id + up_dw_id * 61 * 8 + disk_id * 61 * 8 * 2
    else:
        seq_id = 61 * 8 * 2 * 2 + sc_id + up_dw_id * 61 + disk_id * 61 * 2
    return int(seq_id)

compute_seq_id = np.vectorize(compute_seq_id, otypes=[int])

In [7]:
def get_hit_info_df(f, df, event):
    """Get the hit info for an event in a pandas dataframe
    
    Parameters
    ----------
    f : HDF5 file
        The RICHAI HDF5 file.
    df : pandas DataFrame
        A pandas DataFrame representation of the HDF5 Events file.
    event : int
        The event number to get the hit info for.
        
    Returns
    -------
    positions : pd.DataFrame
        pandas DataFrame with hits data for a given event
    """
    positions = []
    
    # get our hit data for this event
    idx_from = df.loc[event]["first_hit"]
    idx_to = df.loc[event]["last_hit"]
    hit_data = f["Hits"][idx_from:idx_to]
    
    # get our pm locations for this event
    for hit in hit_data:
        pm_idx = compute_seq_id(hit)
        positions.append(position_map[pm_idx])
    
    # add hit time, chod time, and delta
    positions = pd.DataFrame(positions, columns=["x", "y", "mirror"])
    positions["hit_time"] = hit_data["hit_time"]
    positions["chod_time"] = df["chod_time"][event]
    positions["chod_delta"] = positions["hit_time"] - positions["chod_time"]
    positions["class"] = df["class"][event]
    positions["event"] = event
    
    return positions

In [8]:
def draw_pmt_pos(ax,pmt_pos):
    """
        Add circle patches corresponding to the PMT position to the Axes object ax
    """
    for i in pmt_pos:
        if i[2] == 0: # 0: Jura / 1: Salève, PMT disks are identical, we can pick either one. [TODO: CHECK!]
            ax.add_patch(plt.Circle((i[0],i[1]),1.0, color='black'))
    return ax

In [9]:
def get_class_samples(df, n, seed, momentum_bin=None):
    """Sample n samples for each particle class from the events dataframe"""
    
    if momentum_bin is not None:
        df = df.query("momentum_bin == @momentum_bin")
        
    samples = pd.concat(
        [
            df[df["class"] == "muon"].sample(n=n, random_state=seed),
            df[df["class"] == "pion"].sample(n=n, random_state=seed),
            df[df["class"] == "positron"].sample(n=n, random_state=seed)        
        ]

    )
    
    return samples

In [10]:
file_paths = glob.glob("/data/bvelghe/capstone2022/*/*")
data_files = read_RICHAI_data(file_paths)
data_files.keys()

dict_keys(['A/Run008563', 'A/Run008548', 'A/Run008564', 'A/Run008553', 'A/Run008562', 'C/2018E', 'B/2018B'])

In [16]:
f = data_files["A/Run008563"]
f.keys()

<KeysViewHDF5 ['Events', 'HitMapping', 'Hits']>

In [13]:
position_map = np.load("../tools/rich_pmt_positions.npy")
position_map.shape

(1952, 3)

In [77]:
df = events_to_pandas(f)

# add class label (text)
df["class"] = df["label"]
# .apply(get_string_label)

# add momentum bin
momentum_bins = ['0-9', '10-19', '20-29', '30-39','40+']
df["momentum_bin"] = pd.cut(
    df["track_momentum"],
    [0, 10, 20, 30, 40, np.inf],
    labels=momentum_bins
)

df.shape

(181847, 18)

In [78]:
df

,run_id,burst_id,event_id,track_id,track_momentum,chod_time,ring_radius,ring_centre_pos_x,ring_centre_pos_y,ring_likelihood_pion,ring_likelihood_muon,ring_likelihood_positron,label,first_hit,last_hit,total_hits,class,momentum_bin
0,8563,1502,19187,0,29.086382,24.875072,181.268814,-4.406287,31.420258,1.229546e-08,1.000000e+00,9.656426e-02,0,0,16,16,0,20-29
1,8563,1502,43695,0,29.131575,23.019239,180.360535,-33.380520,81.289101,2.874574e-05,1.000000e+00,1.647658e-09,0,16,38,22,0,20-29
2,8563,1502,53258,0,53.725155,7.871216,187.981369,-124.974808,-26.034964,3.173104e-01,1.000000e+00,9.871307e-02,0,38,68,30,0,40+
3,8563,1502,79840,1,45.426033,15.969522,181.346909,-159.838486,-44.940845,1.382166e-01,1.000000e+00,2.374848e-01,0,68,109,41,0,40+
4,8563,1502,88854,0,51.754585,23.295773,186.581955,-131.870239,-39.216457,7.380376e-01,1.000000e+00,5.661391e-02,0,109,134,25,0,40+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181842,8563,6,1901718,0,9.632858,19.408539,191.426300,-341.561005,-60.012337,1.216099e-37,1.216099e-37,1.000000e+00,2,82065,82087,22,2,0-9
181843,8563,7,1153616,0,29.055141,18.719345,189.682892,-217.117447,-70.349503,6.483635e-36,8.388432e-11,1.000000e+00,2,82087,82129,42,2,20-29
181844,8563,8,998857,0,28.937969,5.329090,188.283539,-201.229034,66.584831,1.116412e-17,1.061459e-04,1.000000e+00,2,82129,82196,67,2,20-29
181845,8563,8,1107423,0,27.879078,1.462662,192.117065,-120.966507,47.340836,2.396222e-32,3.192111e-11,1.000000e+00,2,82196,82214,18,2,20-29


In [79]:
# drop the rows having ring radius >=  999999
df_clean = df[df["ring_radius"] < 999999.000]
print(f"Rows removed = {df_clean.shape[0] - df.shape[0]}")

Rows removed = -306


In [80]:
df_clean.describe()

,run_id,burst_id,event_id,track_id,track_momentum,chod_time,ring_radius,ring_centre_pos_x,ring_centre_pos_y,ring_likelihood_pion,ring_likelihood_muon,ring_likelihood_positron,label,first_hit,last_hit,total_hits,class
count,181541.0,181541.000000,1.815410e+05,181541.000000,181541.000000,181541.000000,181541.000000,181541.000000,181541.000000,1.815410e+05,1.815410e+05,1.815410e+05,181541.000000,1.815410e+05,1.815410e+05,1.815410e+05,181541.000000
mean,8563.0,756.529153,1.048428e+06,0.085931,38.018307,13.883745,180.231552,5039.194336,5134.444824,2.820930e-01,8.129908e-01,1.668975e-01,0.115902,2.739948e+06,2.739986e+06,3.768837e+01,0.115902
std,0.0,433.008120,5.936956e+05,0.298646,12.893655,8.259091,12.498135,71473.531250,71466.687500,3.918183e-01,3.473901e-01,3.203971e-01,0.349067,1.861994e+06,1.862008e+06,2.142777e+04,0.349067
min,8563.0,1.000000,9.987000e+03,0.000000,6.286640,-24.252262,0.000000,-501.250580,-287.492249,1.216099e-37,1.216099e-37,1.216099e-37,0.000000,0.000000e+00,0.000000e+00,-6.081777e+06,0.000000
25%,8563.0,378.000000,5.357320e+05,0.000000,28.505917,7.585523,178.524704,-150.765579,-60.075806,1.246960e-05,8.445922e-01,4.466343e-10,0.000000,9.576830e+05,9.577830e+05,2.200000e+01,0.000000
50%,8563.0,766.000000,1.053023e+06,0.000000,37.176815,14.063820,183.695877,-86.141121,0.977586,3.238345e-02,1.000000e+00,7.126957e-04,0.000000,2.663479e+06,2.663507e+06,3.300000e+01,0.000000
75%,8563.0,1122.000000,1.560227e+06,0.000000,47.050396,20.597366,186.824081,-26.465040,63.193901,5.156801e-01,1.000000e+00,1.257385e-01,0.000000,4.368751e+06,4.368799e+06,4.800000e+01,0.000000
max,8563.0,1505.000000,2.166032e+06,6.000000,74.302505,40.877220,429.784088,999999.000000,999999.000000,1.000000e+00,1.000000e+00,1.000000e+00,2.000000,6.081777e+06,6.856199e+06,6.773985e+06,2.000000


In [81]:
# removing last observation as an outlier sentinel
df_clean[df_clean["ring_centre_pos_x"]<999999].describe()

,run_id,burst_id,event_id,track_id,track_momentum,chod_time,ring_radius,ring_centre_pos_x,ring_centre_pos_y,ring_likelihood_pion,ring_likelihood_muon,ring_likelihood_positron,label,first_hit,last_hit,total_hits,class
count,180609.0,180609.000000,1.806090e+05,180609.000000,180609.000000,180609.000000,180609.000000,180609.000000,180609.000000,1.806090e+05,1.806090e+05,1.806090e+05,180609.000000,1.806090e+05,1.806090e+05,1.806090e+05,180609.000000
mean,8563.0,756.423788,1.048449e+06,0.085727,38.092136,13.879004,180.397491,-95.115456,0.627115,2.821668e-01,8.136184e-01,1.667958e-01,0.115426,2.741675e+06,2.741713e+06,3.772583e+01,0.115426
std,0.0,433.010563,5.937261e+05,0.298269,12.852130,8.257301,11.905288,75.115715,75.968132,3.916720e-01,3.468123e-01,3.201186e-01,0.348448,1.861729e+06,1.861744e+06,2.148299e+04,0.348448
min,8563.0,1.000000,9.991000e+03,0.000000,7.939959,-24.252262,31.765896,-501.250580,-287.492249,1.216099e-37,1.216099e-37,1.216099e-37,0.000000,0.000000e+00,0.000000e+00,-6.081777e+06,0.000000
25%,8563.0,378.000000,5.356000e+05,0.000000,28.585278,7.578152,178.598816,-151.070251,-60.451252,1.334817e-05,8.482141e-01,5.262062e-10,0.000000,9.605260e+05,9.605510e+05,2.200000e+01,0.000000
50%,8563.0,766.000000,1.053069e+06,0.000000,37.241077,14.059467,183.717499,-86.828590,0.259079,3.278814e-02,1.000000e+00,7.311064e-04,0.000000,2.666284e+06,2.666295e+06,3.300000e+01,0.000000
75%,8563.0,1122.000000,1.560279e+06,0.000000,47.096119,20.590668,186.831665,-27.198938,62.199364,5.155568e-01,1.000000e+00,1.260061e-01,0.000000,4.370116e+06,4.370164e+06,4.800000e+01,0.000000
max,8563.0,1505.000000,2.166032e+06,6.000000,74.302505,40.877220,429.784088,152.683441,240.800430,1.000000e+00,1.000000e+00,1.000000e+00,2.000000,6.081777e+06,6.856199e+06,6.773985e+06,2.000000


In [82]:
df_clean[df_clean["ring_radius"]>300]

,run_id,burst_id,event_id,track_id,track_momentum,chod_time,ring_radius,ring_centre_pos_x,ring_centre_pos_y,ring_likelihood_pion,ring_likelihood_muon,ring_likelihood_positron,label,first_hit,last_hit,total_hits,class,momentum_bin
51243,8563,1031,1071651,0,45.490204,20.155180,324.840576,152.683441,24.590130,0.016205,0.229713,1.000000,0,1898023,1898068,45,0,40+
162011,8563,7,194272,0,56.790730,19.333471,429.784088,-150.813705,-287.492249,0.995475,1.000000,0.443699,0,6064181,6064189,8,0,40+


In [147]:
# Removing outlier radii >300
df_clean = df_clean[(df_clean["ring_radius"]<300) & (df_clean["ring_radius"]>0)]

In [148]:
# Cleaning for total hits
df_clean = df_clean[df_clean['total_hits']>0]

In [176]:
# Cleaning for anomalies in ring_center_pos_X
df_clean = df_clean[df_clean['ring_centre_pos_x']<1000]

In [177]:
df_clean.shape

(180605, 18)

## Training LightGBM

In [150]:
df_clean.columns

Index(['run_id', 'burst_id', 'event_id', 'track_id', 'track_momentum',
       'chod_time', 'ring_radius', 'ring_centre_pos_x', 'ring_centre_pos_y',
       'ring_likelihood_pion', 'ring_likelihood_muon',
       'ring_likelihood_positron', 'label', 'first_hit', 'last_hit',
       'total_hits', 'class', 'momentum_bin'],
      dtype='object')

In [178]:
# Defining X and y
X = df_clean.loc[:, [
    'track_momentum',
    'ring_radius',
    'total_hits'
]]

y = df_clean.loc[:,'class']

In [179]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    stratify=y, 
    test_size=0.25
)

In [153]:
# Training function
def mean_std_cross_val_scores(model, X_train, y_train, **kwargs):
    """
    Returns mean and std of cross validation

    Parameters
    ----------
    model :
        scikit-learn model
    X_train : numpy array or pandas DataFrame
        X in the training data
    y_train :
        y in the training data

    Returns
    ----------
        pandas Series with mean scores from cross_validation
    """

    scores = cross_validate(model, X_train, y_train, **kwargs)

    mean_scores = pd.DataFrame(scores).mean()
    std_scores = pd.DataFrame(scores).std()
    out_col = []

    for i in range(len(mean_scores)):
        out_col.append((f"%0.3f (+/- %0.3f)" % (mean_scores[i], std_scores[i])))

    return pd.Series(data=out_col, index=mean_scores.index)

In [154]:
# Scoring metrics
scoring_metrics = [
    "accuracy",
    "f1",
    "recall",
    "precision",
    "roc_auc",
    "average_precision"
]

In [155]:
df_clean.columns

Index(['run_id', 'burst_id', 'event_id', 'track_id', 'track_momentum',
       'chod_time', 'ring_radius', 'ring_centre_pos_x', 'ring_centre_pos_y',
       'ring_likelihood_pion', 'ring_likelihood_muon',
       'ring_likelihood_positron', 'label', 'first_hit', 'last_hit',
       'total_hits', 'class', 'momentum_bin'],
      dtype='object')

In [156]:
# features
numeric_features = [
    'track_momentum',
    'ring_radius',
    'total_hits'
]
drop_features = ['run_id', 'burst_id', 'event_id', 'track_id', 'momentum_bin']

In [180]:
# Preprocessor
preprocessor = make_column_transformer(
    (StandardScaler(), numeric_features),
    ("drop", drop_features)
)

In [181]:
# Pipeline
results = defaultdict(list)
pipe_xgb = make_pipeline(
    preprocessor, XGBClassifier(random_state=123, eval_metric="logloss", verbosity=0)
)
pipe_lgbm = make_pipeline(preprocessor, LGBMClassifier(
    random_state=123, 
    class_weight='balanced'
    )
)

pipe_catboost = make_pipeline(
    preprocessor, CatBoostClassifier(verbose=0, random_state=123)
)
classifiers = {
#     "XGBoost": pipe_xgb,
    "LightGBM": pipe_lgbm,
#     "CatBoost": pipe_catboost,
}

In [182]:
y_train.value_counts(normalize=True)

0    0.894236
1    0.096107
2    0.009656
Name: class, dtype: float64

In [183]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180605 entries, 0 to 181846
Data columns (total 18 columns):
 #   Column                    Non-Null Count   Dtype   
---  ------                    --------------   -----   
 0   run_id                    180605 non-null  int32   
 1   burst_id                  180605 non-null  int32   
 2   event_id                  180605 non-null  int64   
 3   track_id                  180605 non-null  int32   
 4   track_momentum            180605 non-null  float32 
 5   chod_time                 180605 non-null  float32 
 6   ring_radius               180605 non-null  float32 
 7   ring_centre_pos_x         180605 non-null  float32 
 8   ring_centre_pos_y         180605 non-null  float32 
 9   ring_likelihood_pion      180605 non-null  float32 
 10  ring_likelihood_muon      180605 non-null  float32 
 11  ring_likelihood_positron  180605 non-null  float32 
 12  label                     180605 non-null  int32   
 13  first_hit                 180

In [184]:
y_train.shape

(135453,)

In [185]:
# training
for (name, model) in classifiers.items():
    results[name] = mean_std_cross_val_scores(
        model, X_train, y_train, 
        return_train_score=True, 
        scoring=scoring_metrics, 
        n_jobs=-1
    )

KeyboardInterrupt: 

In [ ]:
# pd.DataFrame(results)

In [ ]:
# Didnt run. Further exploring implementation of LightGBM GPU training/ Apache Spark - MLFlow
https://lightgbm.readthedocs.io/en/latest/GPU-Tutorial.html 
https://lightgbm.readthedocs.io/en/latest/Parallel-Learning-Guide.html